In [121]:
# 화면 비율 넓게 설정
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max.colwidth',150)

C:\Users\정승우\AppData\Local\Temp\ipykernel_20052\1083280195.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [124]:
# 다나와 인텔 CPU 크롤링 코드
from bs4 import BeautifulSoup
import urllib.request
import re

#웹브라우져 헤더 추가
#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

intel_link ='https://prod.danawa.com/list/?cate=113973'
amd_link = 'https://prod.danawa.com/list/?cate=113990'

def crawling(link):
    req = urllib.request.Request(link, headers = hdr)
    data = urllib.request.urlopen(req).read()
    page = data.decode('utf-8','ignore')
    soup = BeautifulSoup(page, 'html.parser')
    crolled_data = soup.find_all('div', attrs={'class':'prod_main_info'})
    # print(crolled_data)
    # 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
    i=0
    partslist = list()
    for item in crolled_data:
        if i>=4 and i!=9:
            if i==30: break;
            # strip 메서드로 앞 뒤 공백 제거
            name = item.select_one('p > a').get_text()[8:].strip() # 부품명
            price = item.select_one('p > a > strong').get_text() # 가격
            # print(item.select_one('div.spec-box.spec-box--full > div.spec_list > a:nth-child(1)').get_text()) # 소캣
            #productItem16100921 > div > div.prod_info > div.spec-box.spec-box--full > div.spec_list > span:nth-child(31 또는 28) > a
            # 내장그래픽 탑재 여부가 원래 31번째 있지만 어떤 애들은 28번째에 있어서 에러나서 split 한 다음 
            details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
            socket = details[0].strip() if details else 'N/A' # 소캣
            integrated_graphics = True if '내장그래픽: 탑재' in details else False # 내장그래픽
            for detail in details:
                if '메모리 규격' in detail:
                    memory_type = str(detail[8:])
                    
                if '시네벤치R23(싱글)' in detail:
                    singlebench = detail[13:]
                    
                if '시네벤치R23(멀티)' in detail:
                    multibench = detail[13:]
                    
            # print([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
            partslist.append([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Socket', 'Integrated Graphics','메모리 타입','시네벤치R23(싱글)','시네벤치R23(멀티)'])
    return result
intel = crawling(intel_link)
amd = crawling(amd_link)

In [128]:
intel
intel.to_csv('intel.csv',encoding='cp949', index=False)

In [129]:
amd
amd.to_csv('amd.csv',encoding='cp949', index=False)

In [33]:
# 다나와 인텔 CPU 가격만 크롤링 코드
from bs4 import BeautifulSoup
import urllib.request
import re

#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
data ='https://prod.danawa.com/list/?cate=113973'
#웹브라우져 헤더 추가
req = urllib.request.Request(data, \
                            headers = hdr)

data = urllib.request.urlopen(req).read()
page = data.decode('utf-8','ignore')
soup = BeautifulSoup(page, 'html.parser')
crolled_data = soup.find_all(['p','strong'],attrs={'class':'price_sect'})

# 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
i=0
partslist = list()
for item in crolled_data:
    if i>=4 and i!=9:
        # strip 메서드로 앞 뒤 공백 제거
        print(item.select_one('strong').get_text())
    i+=1
print(partslist)

# add_csv('cpu', partslist)

588,390
568,160
536,000
283,730
274,590
300,240
147,990
133,870
122,160
151,680
813,680
797,520
646,290
346,700
348,990
341,790
399,220
191,890
388,990
396,920
387,370
420,170
550,040
511,570
612,890
238,000
226,000
212,660
247,940
445,790
428,240
541,170
522,460
500,000
511,790
601,640
453,410
460,750
451,310
584,140
551,540
568,460
539,000
758,990
208,040
174,110
194,560
105,900
95,470
85,550
101,440
395,290
371,290
466,500
152,330
137,270
127,590
179,900
772,650
767,090
921,640
368,990
335,120
325,670
387,200
201,260
175,920
165,230
283,300
124,990
98,340
79,000
203,500
360,100
297,980
286,790
513,340
92,360
85,100
80,190
75,230
219,700
292,000
259,990
242,270
243,000
322,000
504,290
507,460
498,020
636,540
182,150
132,290
127,970
194,380
969,380
411,930
333,260
343,330
327,930
445,700
234,140
202,380
211,470
224,290
339,980
325,040
272,260
402,900
324,940
285,730
272,080
201362
[]


In [139]:
# 램 가격 크롤링
from bs4 import BeautifulSoup
import urllib.request
import re

#웹브라우져 헤더 추가
#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

ddr4_link = 'https://prod.danawa.com/list/?cate=1131326&searchOption=/innerSearchKeyword='
ddr5_link = 'https://prod.danawa.com/list/?cate=11341201&searchOption=/innerSearchKeyword='

def crawling(link):
    req = urllib.request.Request(link, headers = hdr)
    data = urllib.request.urlopen(req).read()
    page = data.decode('utf-8','ignore')
    soup = BeautifulSoup(page, 'html.parser')
    crolled_data = soup.find_all('div', attrs={'class':'prod_main_info'})
    # print(crolled_data)
    # 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
    i=0
    partslist = list()
    for item in crolled_data:
        if i>=4 and i!=9:
            if i==30: break;
            # strip 메서드로 앞 뒤 공백 제거
            name = item.select_one('p > a').get_text()[8:].strip() # 부품명
            price = item.select_one('p > a > strong').get_text() # 가격
            details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
            speed = details[2]
            partslist.append([name, price, speed])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price','Speed'])
    return result
ddr4 = crawling(ddr4_link)
ddr5 = crawling(ddr5_link)

In [142]:
ddr4
ddr4.to_csv('ddr4.csv',encoding='cp949', index=False)

In [143]:
ddr5
ddr5.to_csv('ddr5.csv',encoding='cp949', index=False)